# 被管理的属性

## property

property协议允许我们把一个特定属性的获取、设置和修改操作指向我们所提供的函数或方法，使得我们能够插入在属性访问时自动运行的代码，或是拦截属性的删除

```python
attribute = property(fget, fset, fdel, doc)
```
- fget传入一个函数用于拦截属性访问
- fset传入一个函数用于属性赋值
- fdel传入一个函数用于属性删除
- doc属性用于接收该属性的一个文档字符串，否则property会复制fget的文档字符串，而其默认值也一样是None

In [5]:
class Person:
    def __init__(self, name):
        self._name = name 
    
    def getName(self):
        print('fetch...')
        return self._name
    
    def setName(self, value):
        print('change...')
        self._name = value 
    
    def delName(self):
        print('remove...')
        del self._name 
    
    name = property(getName, setName, delName, 'name property docs')

In [6]:
bob = Person('Bob Smith')
print(bob.name)
bob.name = 'Robert Smith'
print(bob.name)
del bob.name 

fetch...
Bob Smith
change...
fetch...
Robert Smith
remove...


并且该特性可以被继承

### 计算出的属性

当获取属性的时候，动态地计算属性地值。

In [7]:
class PropSquare:
    def __init__(self, start):
        self.value = start 
    
    def getX(self):
        return self.value ** 2 # 每次获取都会根据当前值计算出新的值
    
    def setX(self, value):
        self.value = value 
    
    x = property(getX, setX)

In [8]:
P = PropSquare(3)
Q = PropSquare(32)

In [9]:
print(P.x)
P.x = 4
print(P.x)

9
16


In [12]:
help(property)

Help on class property in module builtins:

class property(object)
 |  property(fget=None, fset=None, fdel=None, doc=None)
 |  
 |  Property attribute.
 |  
 |    fget
 |      function to be used for getting an attribute value
 |    fset
 |      function to be used for setting an attribute value
 |    fdel
 |      function to be used for del'ing an attribute
 |    doc
 |      docstring
 |  
 |  Typical use is to define a managed attribute x:
 |  
 |  class C(object):
 |      def getx(self): return self._x
 |      def setx(self, value): self._x = value
 |      def delx(self): del self._x
 |      x = property(getx, setx, delx, "I'm the 'x' property.")
 |  
 |  Decorators make defining new properties or modifying existing ones easy:
 |  
 |  class C(object):
 |      @property
 |      def x(self):
 |          "I am the 'x' property."
 |          return self._x
 |      @x.setter
 |      def x(self, value):
 |          self._x = value
 |      @x.deleter
 |      def x(self):
 |          del s

In [13]:
 class C(object):
     @property
     def x(self):
         "I am the 'x' property."
         return self._x
     @x.setter
     def x(self, value):
         self._x = value
     @x.deleter
     def x(self):
         del self._x

In [18]:
B=C()
B.x = 10
B.x

10

In [19]:
dir(B)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_x',
 'x']

## 描述符

描述符提供了拦截属性访问的另一种替代方法；property是描述符的一种。描述符协议允许我们把一个特定属性的获取、设置和删除指向我们提供都得一个单独类对象的方法。

描述符编写成单独的类，并且针对向要拦截的属性访问操作提供明确指定的访问器方法。当以相应的方式访问赋值了描述符类实例的属性时，描述符类中的获取、设置和删除等方法会自动运行。
```python
class Descriptor:
    def __get__(self, instance, owner):...
    def __set__(self, instance, value):...
    def __delete__(self, instance):...
```
所有带有这么方法的类都可以看作描述符，并且当它们的一个实例被赋值给另一个类的属性的时候，它们的这些方法就会成为特殊的。这些方法却已不可。。

In [1]:
class Descriptor:
    def __get__(self, instance, owner):
        print(self, instance, owner, sep='\n')

In [2]:
class Subject:
    attr = Descriptor()

In [3]:
X = Subject()
X.attr

<class '__main__.Subject'>


In [4]:
Subject.attr

None
<class '__main__.Subject'>


在第一个属性获取中自动传递到__get__方法中的参数，当获取X.attr的时候，就好像发生了如下的转换：
```python
X.attr -> Descriptor.__get__(Subject.attr, X, Subject)
```
当描述符的instance参数为None的时候，该描述符就知道它正被通过类名直接访问。

### 只读描述符

和property不同的是，在使用描述符时如果忽略描述符中的__set__方法就不能够让属性称为制度的，因为描述名称可以赋给一个实例。

In [5]:
class D:
    def __get__(*args):print('get')

class C:
    a = D()

In [6]:
X = C()
X.a

get


In [7]:
C.a

get


In [8]:
X.a = 99
X.a

99

要想让基于描述符的属性成为只读的，你需要在描述符中捕获赋值操作并引发一个异常来阻止属性赋值。 

In [17]:
class D:
    def __get__(*args):print('get')
    def __set__(*args):raise AttributeError('cannot set')

class C:
    def __init__(self):
        self.b = 10
    a = D()

In [18]:
X = C() 
X.a 
X.b

get


10

In [19]:
X.a = 99

AttributeError: cannot set

In [20]:
X.b = 20
X.b 

20

### 示例

In [21]:
class Name:
    # self是Name类属性，instance是Person类属性，owner是Person类
    def __get__(self, instance, owner):
        print('fetch...')
        return instance._name
    
    def __set__(self, instance, value):
        print('change...')
        instance._name = value 
    
    def __delete__(self, instance):
        print('remove...')
        del instance._name 

In [22]:
class Person:
    def __init__(self, name):
        self._name = name 
    name = Name() # 把描述符赋值给类属性

In [24]:
bob = Person('Bob Smith')
print(bob.name)

fetch...
Bob Smith


In [25]:
bob.name = 'szq'
bob.name 

change...
fetch...


'szq'

和property一样，可以在每次获取属性的时候计算它们的值

### 描述符中使用状态信息

描述符除了能使用存储在客户实例中的数据，也能使用附加到描述符对象本身的数据。

- 描述符状态用来管理描述符内部使用的数据，或是**横跨所有实例的数据**。它可以随着属性形式的不同而不同。。。基于描述符的数据
- 实例状态记录了和客户类相关，或是被客户类创建的信息。它可以随着客户类实例的不同而不同。。。基于客户类实例的数据

In [26]:
class DescState:
    def __init__(self, value):
        self.value = value 
    
    def __get__(self, instance, owner):
        print('DescState get')
        return self.value * 10 # 计算值
    
    def __set__(self, instance, value):
        print('DescState set')
        self.value = value 

In [27]:
class CalcAttrs:
    X = DescState(2) # 描述符状态
    Y = 3
    def __init__(self):
        self.Z = 4

In [28]:
obj = CalcAttrs()
print(obj.X, obj.Y, obj.Z)

DescState get
20 3 4


In [29]:
obj.X = 5
CalcAttrs.Y = 6
obj.Z = 7
print(obj.X, obj.Y, obj.Z)

DescState set
DescState get
50 6 7


In [30]:
obj2 = CalcAttrs()
print(obj2.X, obj2.Y, obj2.Z)

DescState get
50 6 4


因为这段代码内部的value信息仅存在于描述符之中，因此如果在客户类的实例中使用相同的名称，也不会冲突。这里描述符管理的属性只拦截X，可以看到所有实例共享一个X值，因为X是绑定的描述符状态，而不是实例的自建的属性。如果使用实例的属性，则每个实例都有一个自己的属性副本

In [31]:
class InstState:
    def __get__(self, instance, owner):
        print('Instance get')
        return instance._X * 10
    
    def __set__(self, instance, value):
        print('Instance set')
        instance._X = value 

In [32]:
class CalcAttrs:
    X = InstState()
    Y = 3
    def __init__(self):
        self._X = 2
        self.Z = 4

In [33]:
obj = CalcAttrs()
print(obj.X, obj.Y, obj.Z)

Instance get
20 3 4


In [35]:
obj.X = 5
CalcAttrs.Y = 5
obj.Z = 6
print(obj.X, obj.Y, obj.Z)

Instance set
Instance get
50 5 6


In [36]:
obj2 = CalcAttrs()
print(obj2.X, obj2.Y, obj2.Z)

Instance get
20 5 4


描述符相较于Proprty的一个常见的优势就是描述符有自己的状态，所以可以很容易的在内部存储数据，而不用将数据添加到客户实例对象的命名空间中。

### property与描述符

In [37]:
# 可以用描述符实现property
class Property:
    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget 
        self.fset = fset 
        self.fdel = fdel
    
    def __get__(self, instance, instanceType=None):
        if instance is None:
            return self
        if self.fget is None:
            raise AttributeError("can't get attribute")
        return self.fget(instance)
    
    def __set__(self, instance, value):
        if self.fset is None:
            raise AttributeError("can't set attribute")
        self.fset(instance, value)
    
    def __delete__(self, instance):
        if self.fdel is None:
            raise AttributeError("can't delete attribute")
        self.fdel(instance)

## __getattr__ 和 __getattribute__

两个运算符重载方法提供了拦截实例的属性获取的方法。但是这两个能拦截任意的名称。所以可以扮演更多的角色比如代理。

- `__getattr__`针对未定义的属性运行。因为它只能为不存储在实例中或是不继承自它的类的属性运行
- `__getattribute__`针对所有的属性运行，因为它是涵盖一切的，所以在使用它的时候，必须小心避免通过把属性访问传递给父类而导致递归循环。

其中死循环的现象如下所示：
```pyhton
def __getattribute__(self, name):
    x = self.other # 由于getattrubute拦截所有属性获取，继续调用就会进入死循环
```

为了避免死循环，需要将获取指向一个更高的父类，从而跳过这个层级的版本，因为`object`总是一个新式类的父类，所以选择它在这里可以很好的起作用。

```python
def __getattribute__(self, name):
    x = object.__getattribute__(self, 'other')
```

`__setattr__`避免死循环的方式：
```js
def __setattr__(self, name, value):
    self.__dict__['other'] = value 
    object.__setattr__(self, 'other', value)
```

In [9]:
class Person:
    def __init__(self, name):
        self._name = name  # 这一句话也会调用__setattr__的呦
    
    def __getattr__(self, attr):
        print('get: ' + attr)
        if attr == 'name':
            return self._name 
        else:
            raise AttributeError(attr)
    
    # 优先级比getattr高
    # def __getattribute__(self, attr):
    #     print('getattribute:' + attr)
    #     if attr == 'name':
    #         attr = '_name'
    #     return object.__getattribute__(self, attr)
        
    def __setattr__(self, attr, value):
        print('set: ' + attr)
        if attr == 'name':
            attr = '_name'
        self.__dict__[attr] = value 
    
    def __delattr__(self):
        print('del: ' + attr)
        if attr == 'name':
            attr = '_name'
        del self.__dict__[attr]

In [11]:
bob = Person('Bob Smith')
print('-' * 50)
print(bob.name)
bob.name = 'szq'

set: _name
getattribute:__dict__
--------------------------------------------------
getattribute:name
Bob Smith
set: name
getattribute:__dict__


### 计算出的属性

In [12]:
class AttrSquare:
    def __init__(self, start):
        self.value = start 
    
    def __getattribute__(self, attr):
        if attr == 'X':
            return self.value ** 2
        else:
            return object.__getattribute__(self, attr)

        # return object.__getattribute__(self, 'value') ** 2
    
    def __setattr__(self, attr, value):
        if attr == 'X':
            attr = 'value'
        object.__setattr__(self, attr, value)

- 构造函数中的赋值会触发setattr
- getattribute 中的self.value 会再次触发getattribute

### 拦截内置运算属性

对于内置的运算符重载方法，**隐式运行时**，`getattr`和`getattribute`不会进行拦截，比如`__add__`

In [17]:
class GetAttr:
    eggs = 88
    def __init__(self):
        self.spam = 77
    
    def __len__(self):
        print('__len__: 42')
        return 42
    
    def __getattr__(self, attr):
        print('getattr: ' + attr)
        if attr == '__str__':
            return lambda *args: '[Getattr str]'
        else:
            return lambda *args: None

In [18]:
class GetAttribute:
    eggs = 88
    def __init__(self):
        self.spam = 77
    
    def __len__(self):
        print('__len__: 42')
        return 42
    
    def __getattribute__(self, attr):
        print('getattribute: ' + attr)
        if attr == '__str__':
            return lambda *args: '[Getattribute str]'
        else:
            return lambda *args: None

In [20]:
for Class in GetAttr, GetAttribute:
    X = Class()
    X.eggs
    X.spam
    X.other
    len(X)

    try:
        X[0]
    except:
        print('fail []')

    try:
        X + 99
    except:
        print('fail +')
    
    try:
        X() # 隐式调用
    except:
        print('fail ()')    
    X.__call__ # 显式调用
    
    print(X.__str__())
    print(X)

    print('-'*50)

getattr: other
__len__: 42
fail []
fail +
fail ()
getattr: __call__
--------------------------------------------------
getattribute: eggs
getattribute: spam
getattribute: other
__len__: 42
fail []
fail +
fail ()
getattribute: __call__
getattribute: __str__
[Getattribute str]
--------------------------------------------------
